In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 22.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [20, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
cnt = pd.read_excel('data/거래량.xlsx')
cnt.fillna(0, inplace=True)
cnt

,ID,단지명,단지명.1,단지명.2,단지명.3,단지명.4,단지명.5,단지명.6,단지명.7,단지명.8,단지명.9,단지명.10,단지명.11,단지명.12,단지명.13,단지명.14,단지명.15,단지명.16,단지명.17
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,4.0,1.0,2.0,0.0,0.0,1.0,3.0,0.0,0.0,3.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,3.0
1,서울특별시 강남구 개포동 개포우성3차 160,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,서울특별시 강남구 개포동 개포자이 130,2.0,1.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0
3,서울특별시 강남구 개포동 개포자이 170,2.0,1.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0
4,서울특별시 강남구 개포동 개포주공1단지 40,19.0,8.0,8.0,1.0,2.0,38.0,24.0,17.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16285,서울특별시 은평구 수색동 청구 130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16286,서울특별시 은평구 역촌동 역산미르빌 80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16287,서울특별시 은평구 응암동 밝은문 50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16288,서울특별시 은평구 증산동 두산위브 120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
df = pd.read_excel('data/abcd1.xlsx')
df = df.iloc[:,0]
df

0       서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80
1              서울특별시 강남구 개포동 개포주공1단지 40
2              서울특별시 강남구 개포동 개포주공5단지 60
3              서울특별시 강남구 개포동 개포주공5단지 80
4              서울특별시 강남구 개포동 개포주공6단지 50
                     ...               
4086          서울특별시 중랑구 중화동 중화동건영아파트 60
4087             서울특별시 중랑구 중화동 태능에셈빌 80
4088             서울특별시 중랑구 중화동 한신아파트 50
4089             서울특별시 중랑구 중화동 한신아파트 60
4090             서울특별시 중랑구 중화동 한신아파트 80
Name: ID, Length: 4091, dtype: object

In [6]:
dff = pd.merge(df, cnt, left_on='ID', right_on='ID', how='left')
dff

,ID,단지명,단지명.1,단지명.2,단지명.3,단지명.4,단지명.5,단지명.6,단지명.7,단지명.8,단지명.9,단지명.10,단지명.11,단지명.12,단지명.13,단지명.14,단지명.15,단지명.16,단지명.17
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,4.0,1.0,2.0,0.0,0.0,1.0,3.0,0.0,0.0,3.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,3.0
1,서울특별시 강남구 개포동 개포주공1단지 40,19.0,8.0,8.0,1.0,2.0,38.0,24.0,17.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,서울특별시 강남구 개포동 개포주공5단지 60,3.0,2.0,1.0,0.0,1.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
3,서울특별시 강남구 개포동 개포주공5단지 80,0.0,0.0,6.0,1.0,1.0,3.0,3.0,0.0,1.0,1.0,7.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0
4,서울특별시 강남구 개포동 개포주공6단지 50,3.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,서울특별시 중랑구 중화동 중화동건영아파트 60,1.0,1.0,3.0,3.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0
4087,서울특별시 중랑구 중화동 태능에셈빌 80,3.0,4.0,2.0,0.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0
4088,서울특별시 중랑구 중화동 한신아파트 50,8.0,5.0,7.0,4.0,5.0,7.0,7.0,6.0,6.0,5.0,5.0,4.0,6.0,3.0,4.0,1.0,0.0,0.0
4089,서울특별시 중랑구 중화동 한신아파트 60,26.0,8.0,14.0,5.0,1.0,11.0,14.0,20.0,15.0,13.0,10.0,11.0,16.0,12.0,7.0,1.0,1.0,0.0


In [7]:
dff.to_excel('data/거래량123.xlsx')